# COVID-19 Dashboard
Please find below the COVID-19 Dashboard. <br>
Data is taken from [Public Health England](https://www.gov.uk/government/organisations/public-health-england)<br> 
<br>
The button below is able to update the data on demand to the latest data

In [1]:
# Importing the COVID API and other required dependencies
from uk_covid19 import Cov19API
import json
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipywidgets as wdg
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
def refreshgraph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other 
    scale.value=current 


In [2]:
# defining the update of the hospital and ITU bed graph with new data
def updatehospitu(self):
    filters = [
        'areaType=overview' # taking data from the whole country
    ]
    #taking the hospital beds and ITU beds per day
    structure = { 
        "date": "date",
        "HospitalBeds": "hospitalCases",
        "ITUBeds" : "covidOccupiedMVBeds"
    }
    api = Cov19API(filters=filters, structure=structure)
    hospanditu=api.get_json() # calling the API
    # writing the data out into a JSON file
    with open("hospanditu.json", "wt") as OUTF:
        json.dump(hospanditu, OUTF) 
    # wrangling the data into out DataFrame
    with open("hospanditu.json", "rt") as INFILE:
        rawdata=json.load(INFILE)
    # extracting the dates and getting a start and end date
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    # creating the dataframe filling in any missing dates
    index=pd.date_range(startdate, enddate, freq='D')
    hospitudf=pd.DataFrame(index=index, columns=['HospitalBeds', 'ITUBeds'])
    # Filling the dataframe with data
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['HospitalBeds', 'ITUBeds']:
            if pd.isna(hospitudf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                hospitudf.loc[date, column]=value
    hospitudf.fillna(0.0, inplace=True)


In [3]:
# Creating a button to update data on demand
def access_api(button):
    # changing the icon and disabling button to show to user something has happened
    apibutton.icon="sync"
    apibutton.disabled=True
    apibutton.description="Updating Graph"
    rawdata=callhospanditu()
    apibutton.description="20%" # progress bar
    global hospitudf
    hospitudf=wranglehospanditu(rawdata)
    apibutton.description="40%"
    rawdata=callcovidandvacc()
    apibutton.description="60%"
    global COVIDandVacc
    COVIDandVacc=wranglecovidandvacc(rawdata)
    apibutton.description="80%"
    # forcing the graph to redraw
    refreshgraph()
    apibutton.description="90%"
    # changing the icon back to how it was before to show user action has finished
    apibutton.icon="refresh"
    apibutton.disabled=False # allowing the user to refresh again
    apibutton.description="Update Graphs"


apibutton=wdg.Button(
    description='Update Graphs',
    disabled=False,
    button_style='info',
    tooltip='Click to update with latest PHE data',
    icon='refresh'
)
apibutton.on_click(access_api)

display(apibutton)


Button(button_style='info', description='Update Graphs', icon='refresh', style=ButtonStyle(), tooltip='Click t…

The first graph is the graph of hospital beds and ITU beds used by COVID-19 patients over time.
Using the box on the left you can select to see either just the hospital beds, the ITU beds or both
The box on the right hand side allows you to change from a linear scale to a logarithmic one and vice versa.

In [4]:
# creating the calling funciton 
# creating the structure for the API
def callhospanditu():
    filters = [
        'areaType=overview' # taking data from the whole country
    ]
    #taking the hospital beds and ITU beds per day
    structure = { 
        "date": "date",
        "HospitalBeds": "hospitalCases",
        "ITUBeds" : "covidOccupiedMVBeds"
    }
    api = Cov19API(filters=filters, structure=structure)
    rawdata=api.get_json()
    return rawdata

def callcovidandvacc():
# creating the structure for the API
    filters = [
        'areaType=overview' # taking data from the whole country
    ]
    #taking the COVID cases and cumulative vaccination numbers (had to use publish date as vaccination date stops at March)
    structure = { 
        "date": "date",
        "New COVID Cases": "newCasesByPublishDate",
        "Total 1st Dose" : "cumPeopleVaccinatedFirstDoseByPublishDate",
        "Total 2nd Dose":"cumPeopleVaccinatedSecondDoseByPublishDate",
        "Total Boosted":"cumPeopleVaccinatedThirdInjectionByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    rawdata=api.get_json()
    return rawdata

# creating functions to get rawdata
def wranglehospanditu(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    # creating the dataframe filling in any missing dates
    index=pd.date_range(startdate, enddate, freq='D')
    hospitudf=pd.DataFrame(index=index, columns=['HospitalBeds', 'ITUBeds'])
    # Filling the dataframe with data
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['HospitalBeds', 'ITUBeds']:
            if pd.isna(hospitudf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                hospitudf.loc[date, column]=value
    hospitudf.fillna(0.0, inplace=True)
    return hospitudf

def wranglecovidandvacc(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    # creating the dataframe filling in any missing dates
    index=pd.date_range(startdate, enddate, freq='D')
    COVIDandVacc=pd.DataFrame(index=index, columns=['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted'])
    # Filling the dataframe with data
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted']:
            if pd.isna(COVIDandVacc.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                COVIDandVacc.loc[date, column]=value
    COVIDandVacc.fillna(0.0, inplace=True)
    # saving the data into a pickle file
    return COVIDandVacc

In [5]:
# wrangling the data into out DataFrame
with open("hospanditu.json", "rt") as INFILE:
    rawdata=json.load(INFILE)
# extracting the dates and getting a start and end date
datalist=rawdata['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

# creating the dataframe filling in any missing dates
index=pd.date_range(startdate, enddate, freq='D')
hospitudf=pd.DataFrame(index=index, columns=['HospitalBeds', 'ITUBeds'])

# Filling the dataframe with data
for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['HospitalBeds', 'ITUBeds']:
        if pd.isna(hospitudf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            hospitudf.loc[date, column]=value

hospitudf.fillna(0.0, inplace=True)


In [6]:
# Creating an interactive table:
dataselect1=wdg.SelectMultiple(
    options=['HospitalBeds', 'ITUBeds'],
    value=['HospitalBeds', 'ITUBeds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['Linear', 'Logarithmic'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([dataselect1, scale])

def hospitu_graph(gcols, gscale):
    if gscale=='Linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        hospitudf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


graph=wdg.interactive_output(hospitu_graph, {'gcols': dataselect1, 'gscale': scale})

display(controls, graph)



Output()

The second graph is the graph of vaccinations compared to new COVID cases.
Using the box on the left you can select to see any of: first, second and booster numbers and COVID cases.
The box on the right hand side allows you to change from a linear scale to a logarithmic one and vice versa, however for this we have selected to start on a logarithmic scale to show the effect of the vaccinations on the new cases.

In [7]:
# Comparing Cases with Vaccination Rates
# wrangling the data into out DataFrame
with open("COVIDandVacc.json", "rt") as INFILE:
    rawdata=json.load(INFILE)
# extracting the dates and getting a start and end date
datalist=rawdata['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

# creating the dataframe filling in any missing dates
index=pd.date_range(startdate, enddate, freq='D')
COVIDandVacc=pd.DataFrame(index=index, columns=['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted'])
# Filling the dataframe with data
for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted']:
        if pd.isna(COVIDandVacc.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            COVIDandVacc.loc[date, column]=value

COVIDandVacc.fillna(0.0, inplace=True)
# saving the data into a pickle file


# Creating an interactive table:
dataselect=wdg.SelectMultiple(
    options=['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted'],
    value=['New COVID Cases', 'Total 1st Dose', 'Total 2nd Dose', 'Total Boosted'],
    rows=4,
    description='Stats:',
    disabled=False
)
scale=wdg.RadioButtons(
    options=['Logarithmic', 'Linear'],
    description='Scale:',
    disabled=False
)
controls=wdg.HBox([dataselect, scale])
def COVIDandVacc_graph(gcols, gscale):
    if gscale=='Linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        COVIDandVacc[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


graph=wdg.interactive_output(COVIDandVacc_graph, {'gcols': dataselect, 'gscale': scale})

display(controls, graph)


Output()

_name redacted for coursework submission_, 2021.
All rights reserved. <br>
Based on UK Government [data](https://coronavirus.data.gov.uk) published by [Public Health England](https://coronavirus.data.gov.uk/details/developers-guide). <br>
Template by [Dr Fabrizio Smeraldi](http://www.eecs.qmul.ac.uk/~fabri/)